In [12]:
"""
BUILT-IN TOOLS (LangChain 1.x)
-----------------------------
• "Built-in tools" are ready-made tools provided by LangChain integrations (often in langchain_community).
• You typically just import the tool class and call .invoke(input) or hand it to an agent.
• Great for quick demos: search, wikipedia, math, etc.

Install (typical):
  pip install -U langchain langchain-openai langchain-community
"""

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

ddg = DuckDuckGoSearchRun()
wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

print(ddg.invoke("LangChain tools vs agents"))
print("\n \n")
print(wiki.invoke("ReAct (artificial intelligence)"))


15 Feb 2025 · This article explores LangChain's Tools and Agents, how they work, and how you can leverage them to build intelligent AI-powered applications. 18 Dec 2025 · A main agent coordinates subagents as tools. All routing passes through the main agent. · Agents transfer control to each other via tool calls. · A single agent ... Patterns · Choosing a pattern · Visual overview 29 Aug 2025 · Create tools · Basic tool definition · Customize tool properties · Custom tool name · Custom tool description · Advanced schema definition · Reserved argument names ... 24 Aug 2025 · The meat (memory, retrieval, planning strategies, error handling, orchestration, coordination between tools/agents) is what makes them useful in the real world. 18 Mar 2025 · The key difference lies in flexibility: chains follow a rigid workflow, while agents adapt their behavior using reasoning and external tools. A chain is like a ...

 

Page: Regulation of artificial intelligence
Summary: Regulation of artificia

In [4]:
"""
PYTHON REPL TOOL (LangChain 1.x)
--------------------------------
• A Python REPL tool lets an LLM (or you) execute Python code and return results.
• Useful for: quick math, data munging, verifying calculations.
• SECURITY WARNING: it can execute arbitrary code (RCE risk). Use ONLY in a sandboxed/local setting.

In LangChain 1.x ecosystem, this is typically available via langchain-experimental.

Install:
  pip install -U langchain-experimental
"""

from langchain_experimental.tools import PythonREPLTool

python_tool = PythonREPLTool()

# Direct invocation (no agent needed)
print(python_tool.invoke("x = 10\nprint(x**2)"))


Python REPL can execute arbitrary code. Use with caution.


100



In [16]:
"""
SERPAPI TOOL (LangChain 1.x)
----------------------------
• SerpAPI is a paid search API that returns search engine results.
• Good for: web search results when you need "real-world" answers (and stable API responses).
• Requires an API key: SERPAPI_API_KEY.

Install:
  pip install -U google-search-results langchain-community

Setup:
  export SERPAPI_API_KEY="your_key"
"""

import os
from langchain_community.utilities import SerpAPIWrapper
from dotenv import load_dotenv

load_dotenv(override=True)

# Make sure your env var is set
assert os.getenv("SERPAPI_API_KEY"), "Set SERPAPI_API_KEY in your environment"

serp = SerpAPIWrapper()

# SerpAPIWrapper returns a text summary of results (depending on wrapper config)
print(serp.run("Latest trends in agentic AI"))


['Reporter Anabelle Nicoud spoke to several experts across AI, security, quantum and beyond to better understand where tech will take us in ...', '4. Agentic AI will still be overhyped but will likely be valuable within five years. ... Last year, like virtually everyone else, we predicted ...', "Augmentation: Today's reality where agents enhance human worker capabilities · Automation: An emerging capability where agents automate tasks ...", 'Twenty-three percent of respondents report their organizations are scaling an agentic AI system somewhere in their enterprises (that is, ...', 'From managing your travel to reinventing retail, autonomous AI agents are moving from theory to reality, promising a new era of proactive digital assistance.', 'Technological Evolution · Increased Autonomy: AI agents becoming more independent and capable of complex decision-making · Enhanced ...', 'Trend #1: Agentic AI · Trend #2: AI Governance Security · Trend #3: Disinformation Security · Trend #4: Post-Q

In [18]:
"""
CUSTOM TOOL with @tool (LangChain 1.x)
--------------------------------------
• Use @tool to turn any typed Python function into a LangChain tool.
• The function signature + type hints become the tool input schema automatically.
• The docstring becomes the tool description (helps the model decide when to call it).
• You can use the tool directly via .invoke(...) OR pass it to an agent.

Reference style (LangChain docs):
  from langchain.tools import tool
"""

from langchain.tools import tool

@tool
def calculate_discount(mrp: float, discount_percent: float) -> float:
    """
    Calculate discounted price from MRP and discount percentage.

    Example:
      mrp=2000, discount_percent=10  -> 1800
    """
    return mrp * (1 - discount_percent / 100)

# Direct tool usage (no agent required)
print(calculate_discount.invoke({"mrp": 2000, "discount_percent": 10}))


1800.0


# Multi-Tool Agents - Study Notes

## Overview
Multi-tool agents are AI systems that can use multiple tools/functions to accomplish complex tasks requiring sequential decision-making and execution.

## Core Components

### 1. Tool Selection (Decide Tool)
**What it means:**
- Agent must choose the appropriate tool from available options based on current context
- Requires understanding of what each tool does and when to use it
- Similar to a person deciding whether to use a calculator, search engine, or database

**Key considerations:**
- Tool descriptions/specifications must be clear
- Agent needs reasoning capability to match task requirements with tool capabilities
- May involve scoring or ranking available tools

**Example flow:**
```
Query: "What's the weather in Paris and book me a flight there?"
→ Decides: Use weather API first, then flight booking API
```

### 2. ReAct Loop (Reasoning + Acting)
**Core concept:**
- **Re**asoning + **Act**ing pattern
- Iterative process where agent alternates between thinking and doing

**Loop structure:**
1. **Thought**: Agent reasons about what to do next
2. **Action**: Agent executes a tool/function
3. **Observation**: Agent receives result from action
4. **Repeat**: Continue until task is complete

**Example ReAct trace:**
```
Thought: I need to find current temperature in London
Action: call_weather_api(city="London")
Observation: Temperature is 15°C, cloudy
Thought: Now I should compare this with Paris
Action: call_weather_api(city="Paris")
Observation: Temperature is 18°C, sunny
Thought: I have enough information to answer
Final Answer: Paris is warmer at 18°C vs London's 15°C
```

**Benefits:**
- Interpretable - can see agent's reasoning
- Error correction - agent can adjust based on observations
- Flexible - handles multi-step problems dynamically

### 3. Planning + Execution
**Two-phase approach:**

**Planning Phase:**
- Agent creates a plan/strategy before acting
- Breaks down complex task into subtasks
- Determines order of operations
- May create a dependency graph

**Execution Phase:**
- Follows the plan step-by-step
- Executes tools in planned sequence
- May adapt plan based on results

**Types of planning:**
- **Static planning**: Create full plan upfront, then execute
- **Dynamic planning**: Create high-level plan, refine as you go
- **Hierarchical planning**: Plans at multiple levels of abstraction

**Example:**
```
Task: "Research competitors and create a comparison report"

PLAN:
1. Search for top 3 competitors
2. For each competitor: fetch website, extract key info
3. Organize data into structured format
4. Generate comparison table
5. Write summary analysis

EXECUTION:
Step 1: web_search("top competitors in X industry")
Step 2a: fetch_website(competitor1_url)
Step 2b: extract_info(webpage_content)
... continues through plan
```

## Integration Patterns

**ReAct vs Planning:**
- ReAct: More flexible, discovers path as it goes
- Planning: More efficient, thinks before acting
- **Hybrid**: Plan high-level strategy, use ReAct for execution

**Tool orchestration:**
- Sequential: Tools used one after another
- Parallel: Multiple tools called simultaneously
- Conditional: Tool choice depends on previous results

## Common Challenges

1. **Infinite loops**: Agent gets stuck repeating same actions
2. **Tool misuse**: Selecting wrong tool for the task
3. **Context management**: Keeping track of state across multiple steps
4. **Error handling**: Dealing with failed tool calls gracefully
5. **Token limits**: Long ReAct traces consume context window

## Implementation Considerations

- Define clear stop conditions
- Limit maximum number of iterations
- Provide good tool descriptions
- Include examples in prompts
- Handle partial failures
- Log reasoning traces for debugging

## Real-World Applications

- Customer service bots (search knowledge base → draft response → send email)
- Research assistants (search → fetch articles → summarize → synthesize)
- Data analysis agents (query database → process data → create visualization)
- Coding assistants (search docs → write code → test → debug)

In [27]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool

# 1) Define tools (recommended style in v1)
@tool
def calculator(expression: str) -> str:
    """Do basic math. Input should be a python-style expression like: 23*89."""
    return str(eval(expression))

@tool
def search_stub(query: str) -> str:
    """Fake search tool (demo). Replace with a real web search tool later."""
    return f"(stub) Top result for: {query}"

# 2) Model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3) Create agent (tool-deciding + loop happens inside)
agent = create_agent(
    model=llm,
    tools=[calculator, search_stub],
    system_prompt="You are a helpful assistant. Decide which tool to use and be concise."
)

# 4) Run
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Search India's GDP and calculate 10% of it."}
    ]
})

print(result)


{'messages': [HumanMessage(content="Search India's GDP and calculate 10% of it.", additional_kwargs={}, response_metadata={}, id='f1fb892d-18b5-4a08-9be0-23f3d68042c5'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 109, 'total_tokens': 162, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-D1e9xuSDpXlLPGhUjD77I4NvIkVdY', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bf19b-b5fe-7762-8b66-8e4b165f6bad-0', tool_calls=[{'name': 'search_stub', 'args': {'query': 'India GDP 2023'}, 'id': 'call_U3kWRSv2EXCAmAxf5tPAoovy', 'type': 'tool_call'}, {'name': 'calculator', 'args': {'expression

In [29]:
print(result["messages"][-1].content)

I found the top result for India's GDP in 2023, but I need the specific value to calculate 10% of it. Currently, the calculation shows 0.0 because I used a placeholder. 

Please provide the GDP value, or I can help you find it if you want.


In [7]:
# pip install -U langchain langchain-openai langgraph

import os
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

from langgraph.prebuilt import create_react_agent

# ✅ Make sure your key is set
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not set"

# 1) Tools (Act)
@tool
def calculator(expr: str) -> str:
    """Evaluate a simple arithmetic expression like '12*7+3'."""
    try:
        return str(eval(expr, {"__builtins__": {}}, {}))  # demo-only
    except Exception as e:
        return f"calc_error: {e}"

@tool
def fake_search(query: str) -> str:
    """Fake search tool for demo."""
    kb = {
        "react": "ReAct = Reasoning + Acting loop: think, call tools, observe, repeat, then answer."
    }
    return kb.get(query.lower(), f"No results for: {query}")

tools = [calculator, fake_search]

# 2) LLM (Reason)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3) ReAct Agent (Reasoning <-> Tools loop)
agent = create_react_agent(llm, tools)

prompt = "Use fake_search for 'react' then compute 12*7+3. Output in 2 lines."

# 4) STREAM and PRINT every step (tool calls + tool outputs + final)
for step in agent.stream(
    {"messages": [HumanMessage(content=prompt)]},
    stream_mode="values",
):
    msgs = step["messages"]
    last = msgs[-1]

    # Print tool calls (Act)
    if getattr(last, "tool_calls", None):
        for tc in last.tool_calls:
            print(f"\n[TOOL CALL] {tc['name']}  args={tc['args']}")

    # Print tool observations and final model text
    if getattr(last, "content", None):
        # tool outputs also come as message content
        print(f"\n[MSG] {last.content}")


C:\Users\LotusBlue\AppData\Local\Temp\ipykernel_27548\4231895168.py:36: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)



[MSG] Use fake_search for 'react' then compute 12*7+3. Output in 2 lines.

[TOOL CALL] fake_search  args={'query': 'react'}

[TOOL CALL] calculator  args={'expr': '12*7+3'}

[MSG] 87

[MSG] ReAct = Reasoning + Acting loop: think, call tools, observe, repeat, then answer.  
The result of 12*7+3 is 87.


In [9]:
# Planning + Execution (Two-phase approach) — LangChain demo
# pip install -U langchain langchain-openai pydantic

import os
from typing import List, Literal, Optional
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not set"

# -------------------------
# 1) Tools (Execution phase will use these)
# -------------------------
@tool
def calculator(expr: str) -> str:
    """Evaluate a simple arithmetic expression like '12*7+3'."""
    try:
        return str(eval(expr, {"__builtins__": {}}, {}))  # demo-only
    except Exception as e:
        return f"calc_error: {e}"

@tool
def fake_search(query: str) -> str:
    """Fake search tool (replace with real search/db later)."""
    kb = {
        "react": "ReAct = Reasoning + Acting: model alternates between thinking, calling tools, and observing results.",
        "planning execution": "Two-phase: (1) plan tasks/steps, (2) execute steps with tools; optionally replan."
    }
    return kb.get(query.lower(), f"No results for: {query}")

TOOLS = {"calculator": calculator, "fake_search": fake_search}

# -------------------------
# 2) Planner output schema (Structured plan)
# -------------------------
class PlanStep(BaseModel):
    action: Literal["tool", "final"] = Field(..., description="Use a tool or produce final response")
    tool_name: Optional[Literal["calculator", "fake_search"]] = None
    tool_input: Optional[str] = None
    output_key: Optional[str] = Field(None, description="Where to store this step's output in memory")
    note: Optional[str] = Field(None, description="Short reason for this step")

class Plan(BaseModel):
    steps: List[PlanStep]

# -------------------------
# 3) Create Planner + Executor LLMs
# -------------------------
planner_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
executor_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

planner = planner_llm.with_structured_output(Plan)

# -------------------------
# 4) Two-phase run: PLAN then EXECUTE
# -------------------------
def run_two_phase(user_request: str) -> str:
    # Phase A: Planning
    plan: Plan = planner.invoke(
        [
            ("system",
             "You are a PLANNER. Create a short step-by-step plan. "
             "Use tools only when needed. Output JSON matching the schema."),
            ("user", user_request),
        ]
    )

    print("\n=== PLAN ===")
    for i, s in enumerate(plan.steps, 1):
        print(f"{i}. {s.action} | tool={s.tool_name} | input={s.tool_input} | key={s.output_key} | note={s.note}")

    # Phase B: Execution
    memory = {}  # store tool outputs by output_key
    observations = []  # keep trace

    for i, step in enumerate(plan.steps, 1):
        if step.action == "tool":
            fn = TOOLS[step.tool_name]
            tool_in = step.tool_input or ""
            tool_out = fn.invoke(tool_in)  # LangChain tool execution

            if step.output_key:
                memory[step.output_key] = tool_out

            observations.append((step.tool_name, tool_in, tool_out))
            print(f"\n[EXEC] step {i}: {step.tool_name}({tool_in}) -> {tool_out}")

        elif step.action == "final":
            # Let the executor write the final answer using the gathered outputs
            final = executor_llm.invoke(
                [
                    ("system",
                     "You are an EXECUTOR. Use the plan results to produce the final answer. "
                     "Be concise and correct."),
                    ("user", f"User request:\n{user_request}\n\nTool results (memory):\n{memory}\n\nObservations:\n{observations}")
                ]
            )
            return final.content

    # If planner forgot a final step, still produce an answer
    fallback = executor_llm.invoke(
        [("user", f"Complete the task using these results:\n{memory}\nObservations:\n{observations}")]
    )
    return fallback.content

# -------------------------
# Example
# -------------------------
if __name__ == "__main__":
    req = "Explain ReAct in 1 line, then compute 12*7+3. Use tools if needed."
    answer = run_two_phase(req)
    print("\n=== FINAL ===")
    print(answer)



=== PLAN ===
1. final | tool=None | input=None | key=react_explanation | note=ReAct is a framework that combines reasoning and acting in AI systems to enhance decision-making.
2. tool | tool=calculator | input=12*7+3 | key=calculation_result | note=Calculating the expression 12*7+3.

=== FINAL ===
ReAct is a framework that combines reasoning and acting in AI systems to enhance decision-making. 

Now, computing 12*7+3:  
12 * 7 = 84  
84 + 3 = 87  

Final answer: 87.


# Agent Limitations & Best Practices: Notes for GenAI Developers

## 1. Avoiding Infinite Loops

### What Are Agent Loops?
Agent loops occur when an AI agent repeatedly executes the same actions without making progress toward completing a task, getting stuck in a cycle of unproductive behavior.

### Common Causes
- **Unclear termination conditions**: The agent doesn't know when to stop trying
- **Failed tool calls**: Repeatedly attempting the same action that keeps failing
- **Circular dependencies**: Task A requires Task B, which requires Task A
- **Lack of progress tracking**: Agent can't detect it's not making headway
- **Overconfident retries**: Assuming the next attempt will succeed without changing approach

### Prevention Strategies
**Set explicit limits**: Define maximum iteration counts for any repeated action (e.g., max 3 retries for failed API calls)

**Implement progress checks**: Track what's been attempted and verify each action creates measurable progress toward the goal

**Use state management**: Maintain a clear record of completed steps, failed attempts, and current position in the workflow

**Define clear exit conditions**: Specify both success conditions (task complete) and failure conditions (give up after X attempts)

**Add circuit breakers**: Detect when the same action repeats too many times and force a different approach or escalation

### Code Example Pattern
```python
max_attempts = 3
attempt_history = set()

for attempt in range(max_attempts):
    action_signature = generate_action_signature(current_state)
    
    if action_signature in attempt_history:
        # Detect we're repeating ourselves
        break
    
    attempt_history.add(action_signature)
    result = execute_action()
    
    if is_successful(result):
        break
```

## 2. Tool Misuse

### What Is Tool Misuse?
Tool misuse occurs when an AI agent uses available tools incorrectly, inefficiently, or inappropriately for the task at hand.

### Common Types of Misuse

**Wrong tool for the job**: Using a web scraper when an API exists, using search when calculation is needed

**Excessive tool calls**: Making 50 API requests when 1 would suffice, repeatedly searching for information already obtained

**Ignoring tool outputs**: Calling a tool but not properly using the returned data

**Parameter errors**: Passing incorrect data types, malformed inputs, or ignoring required parameters

**Sequential when parallel possible**: Making dependent calls in sequence when they could run concurrently

**Not handling errors**: Failing to check if tool execution succeeded before proceeding

### Best Practices

**Tool selection logic**: Implement clear decision trees for which tool to use in which situation

**Validate before execution**: Check that inputs are appropriate before calling tools

**Parse and verify outputs**: Always validate tool responses before using them in subsequent steps

**Implement rate limiting**: Respect API limits and avoid overwhelming external services

**Batch operations**: Combine multiple similar requests into single calls when tools support it

**Error handling**: Wrap tool calls in try-catch blocks and have fallback strategies

### Example Patterns
```python
# Good: Check if tool is appropriate first
def get_weather(location):
    if not is_valid_location(location):
        return "Invalid location provided"
    
    try:
        result = weather_api.call(location)
        if result.status == 'success':
            return parse_weather_data(result)
        else:
            return handle_weather_error(result)
    except APIError as e:
        return fallback_weather_source(location)
```

## 3. Hallucinated Tools

### What Are Hallucinated Tools?
Tool hallucination occurs when an AI agent invents or references tools that don't actually exist in its available toolkit, leading to execution failures and confusion.

### Why This Happens
- **Training data confusion**: The model saw similar tools during training
- **Wishful thinking**: The agent "wants" a tool that would solve the problem perfectly
- **Incomplete tool awareness**: Not properly tracking which tools are actually available
- **Name similarity**: Confusing `get_user_data()` with `fetch_user_info()`

### Detection Strategies

**Strict tool registry**: Maintain an explicit list of available tools and validate against it before execution

**Schema validation**: Verify tool calls match exact function signatures

**Execution gatekeeping**: Have a validation layer that rejects calls to non-existent tools

**Logging and monitoring**: Track attempted tool calls to identify hallucination patterns

### Prevention Approaches

**Clear tool documentation in prompts**: Provide the agent with an explicit, complete list of available tools with their exact names and parameters

**Structured output formats**: Require tool calls to follow strict JSON schemas that can be validated

**Confirmation steps**: Before executing, have the agent confirm the tool exists

**Fallback handling**: When a hallucinated tool is detected, prompt the agent to select from actual available tools

### Implementation Example
```python
AVAILABLE_TOOLS = {
    'web_search': {'params': ['query'], 'required': ['query']},
    'calculator': {'params': ['expression'], 'required': ['expression']},
    'file_read': {'params': ['path'], 'required': ['path']}
}

def validate_tool_call(tool_name, params):
    if tool_name not in AVAILABLE_TOOLS:
        raise ToolNotFoundError(
            f"{tool_name} not available. "
            f"Available tools: {list(AVAILABLE_TOOLS.keys())}"
        )
    
    required = AVAILABLE_TOOLS[tool_name]['required']
    if not all(param in params for param in required):
        raise InvalidParametersError(
            f"Missing required parameters: {required}"
        )
    
    return True
```

## General Best Practices for Agent Development

### Planning Before Acting
- Encourage agents to create execution plans before starting
- Break complex tasks into smaller, verifiable subtasks
- Validate plans against available tools and capabilities

### Observability
- Log all tool calls with timestamps and parameters
- Track decision-making process (why this tool was chosen)
- Monitor execution time and resource usage
- Create dashboards for agent behavior analysis

### Graceful Degradation
- Design agents to fail gracefully when tools are unavailable
- Provide partial results rather than complete failure
- Communicate limitations clearly to users

### Testing Strategies
- Unit test individual tool integrations
- Integration test tool combinations
- Simulate tool failures to test error handling
- Use diverse test cases including edge cases

### Human-in-the-Loop
- For critical operations, require human approval
- Provide override mechanisms for agent decisions
- Create escalation paths when agents are stuck

### Documentation Requirements
- Document all available tools with examples
- Maintain change logs when tools are updated
- Create troubleshooting guides for common issues
- Provide clear error messages that aid debugging

---

## Key Takeaways for GenAI Developers

1. **Always assume things can go wrong**: Design for failure, not just success
2. **Make the invisible visible**: Log, monitor, and instrument everything
3. **Constrain creativity**: Sometimes limiting agent freedom prevents problems
4. **Validate relentlessly**: Check inputs, outputs, and tool availability at every step
5. **Learn from failures**: Analyze what went wrong and update your systems accordingly